# Sesión 06: Comparación de grupos
*"La mitad de mi gasto en publicidad no sirve para nada. El problema es que no sé qué mitad" atribuida a John Wanamaker(1838-1922)*

## 6.1  Pruebas A/B
En el área de negocios, especialmente en innovación digital, es común realizar pruebas A/B para evaluar las respuestas de los usuarios a diferentes alternativas en el diseño de un sitio o una interfaz.  
Entre sus características básicas está que los usuarios son asignados al azar a un grupo de control (A) o a un grupo experimental (B). A cada grupo se les muestra una alternativa diferente y se recopilan diversas métricas. Finalmente, analizando los resultados, se concluye qué alternativa tuvo mejor desempeño.  
En la siguiente liga se muestran varios ejemplos de aplicaciones prácticas:
https://goodui.org/leaks/. Este tipo de experimentos ejemplifica la aplicación de técnicas que veremos en esta sesión relacionadas con la comparación de dos o más grupos

El archivo "cookie_cats.csv" contiene información de 90,189 jugadores. Cuando un jugador instala el juego empieza en el nivel 30, sin embargo, en la empresa que diseñó el juego han propuesto que el jugador empiece en el nivel 40. Para evaluar ambas opciones se diseñó una prueba A/B en la cual los jugadores fueron asignados aleatoriamente para comenzar en el nivel 30 (control) o en el nivel 40 (tratamiento).

Las variables en el dataset son:

- userid: identificador del jugador.
- version: gate30, empieza en el nivel 30; gate40: empieza en el nivel 40.
- sum_gamerounds: número de rondas jugadas durante los primeros 14 días.
- retention_1: el jugador jugó un día después de instalar el juego.
- retention_7_ el jugador jugó después de 7 días de haber instalado el juego.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/b07_cookie_cats.csv')
df.info()

In [ ]:
df.describe()

In [ ]:
# Eliminación de datos atípicos
df = df[df['sum_gamerounds']<1000]

## 6.2 Prueba z para dos proporciones
Realizaremos una prueba z para comparar dos proporciones. Las pruebas de hipótesis pueden ser bilaterales (de dos colas) o unilaterales (de una cola). Utilizaremos una prueba bilateral, por lo tanto, las hipótesis se plantean como:
- H<sub>0</sub>: p=p<sub>0</sub> (Hipótesis nula)
- H<sub>1</sub>: p ≠ p<sub>0</sub> (Hipótesis alternativa)

Para calcular las proporciones conviene obtener una tabla con el conteo de cada caso. Realicemos una tabla de contigencia o tabla cruzada. 

In [ ]:
# Crear una tabla de contingencia
tabla_contingencia = pd.crosstab(df['version'], df['retention_1'])
tabla_contingencia

Utilizaremos estos conteos como datos para realizar la prueba z

In [ ]:
from statsmodels.stats.proportion import proportions_ztest
# Prueba z
valor_z, p_value  = proportions_ztest(tabla_contingencia[True], 
                                      tabla_contingencia.sum(axis=1))

# Imprimir el resultado
print("Valor z:", valor_z)
print("Valor p:", p_value)

En las pruebas estadísticas aplicaremos el criterio del p-valor, esto es, rechazar la hipótesis nula si el p-valor es menor al nivel de significancia (típicamente se utiliza 0.05)

In [ ]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de que hay diferencias significativas en las proporciones.")
else:
    print("No se rechaza la hipótesis nula: no hay suficiente evidencia para afirmar que hay diferencias significativas en las proporciones.")

## 6.3 Prueba t para dos muestras independientes

Los promedios también son una métrica comúnmente utilizada en las pruebas A/B. Comparemos las rondas de juego en promedio por cada grupo en el experimento.

In [ ]:
df.groupby('version').agg({'sum_gamerounds':'mean'})

In [ ]:
from scipy.stats import ttest_ind

# Realizar la prueba t de dos muestras independientes
t_statistic, p_value = ttest_ind(df.loc[df['version'] == 'gate_30', 'sum_gamerounds'], 
                                 df.loc[df['version'] == 'gate_40', 'sum_gamerounds'])

# Imprimir los resultados
print("Estadístico t:", t_statistic)
print("Valor p:", p_value)

In [ ]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las medias.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en las medias.")

## 6.4 Prueba Shapiro Wilk

Si la muestra es pequeña (p<30) se requiere evaluar la normalidad en la distribución de los datos. Existen varias pruebas para evaluar la normalidad de los datos. La prueba Shapiro-Wilk es una de las más utilizadas.  
- Hipótesis nula (H0): Los datos provienen de una distribución normal  
- Hipótesis alternativa (H1): Los datos no provienen de una distribución normal.  
Solo como ejercicio, aplicaremos la prueba Shapiro-Wilk

In [ ]:
from scipy.stats import shapiro

df2 = df.head(100)

stat, p_value = shapiro(df2.loc[df2['version'] == 'gate_30', 'sum_gamerounds'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

In [ ]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula de normalidad en los datos")
else:
    print("No se puede rechazar la hipótesis nula de normalidad en los datos")

## 6.5 Prueba de Levene
Es recomendable también comparar la varianza entre ambos grupos. Para ello se realiza la prueba de Levene (prueba de homogeneidad de varianzas).

In [ ]:
from scipy.stats import levene
stat, p_value = levene(df.loc[df['version'] == 'gate_30', 'sum_gamerounds'],
                       df.loc[df['version'] == 'gate_40', 'sum_gamerounds'])
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

In [ ]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las varianzas.")
else:
    print("No se puede rechazar la hipótesis nula: no se rechaza el supuesto de igualdad de varianzas.")

## 6.6. Prueba Mann-Whitney
Una alternativa no paramétrica a la prueba t es la prueba de Mann-Whitney. Esto es recomendable cuando no se cumple el supuesto de normalidad de los datos.

In [ ]:
from scipy.stats import mannwhitneyu

# Realizar la prueba de Mann-Whitney U
statistic, p_value = mannwhitneyu(df.loc[df['version'] == 'gate_30', 'sum_gamerounds'],
                                  df.loc[df['version'] == 'gate_40', 'sum_gamerounds'])

# Imprimir los resultados
print("Estadístico de prueba U:", statistic)
print("Valor p:", p_value)

In [ ]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en la distribución.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en la distribución")

## Lectura
https://roirevolution.com/blog/why-ab-testing-could-save-your-marketing-strategy/